## REI602M Machine Learning - Homework 4
### Due: *Sunday* 14.2.2021

**Objectives**: Parameter tuning, Ensemble tree classifiers, Feature importance, Stacking, Pre-processing, performance metrics, scikit-learn and pandas.

**Name**: Alexander Guðmundsson, **email: ** alg35@hi.is, **collaborators:** (if any)

Please provide your solutions by filling in the appropriate cells in this notebook, creating new cells as needed. Hand in your solution on Gradescope, taking care to locate the appropriate page numbers in the PDF document. Make sure that you are familiar with the course rules on collaboration (encouraged) and copying (very, very, bad).

### 1\. [Pre-processing and parameter tuning in an SVM classifier, 20 points]
The Statlog data set is an old benchmark in machine learning. It contains data from satellite images and the aim is to predict land type (red soil, cotton crop etc). There are 36 integer valued features and 6 classes. The file `sat.trn` contains the 4435 training examples and `sat.tst` contains 2000 test examples. Your task is to obtain an RBF-SVM classifier which obtains high classification accuracy on this data set.

a) Evaluate the accuracy of a RBF-SVM obtained with default values for $C$ and $\gamma$.

b) Scale the training set prior to training an RBF-SVM (scale the test data accordingly) and repeat the task from a).

c) Use cross-validation on the (scaled) training set to identify optimal values of $C$ and $\gamma$. You should start with a coarse grid and then do another run with a finer grid. Logarithmically spaced grid values are often used. Evaluate  the accuracy of the resulting classifier by re-training using the best parameter values and report the error on the test set (why is the cross-validation error not a good estimate of the true classifier error here?)

d) Using randomized search for hyper-parameter values has been shown to be more efficient than traditional grid search. Instead of evaluating parameter values at regular intervals, the values are sampled from a distribution over possible parameter values. This enables considerable time savings (exhaustive grid search is expensive), or the identification of better parameter values for a given computational budget. Repeat the parameter search using `RandomizedSearchCV`. You can either fix the budget (n_iter parameter) so that the cost is comparable to the exhaustive grid search in c), or set the budget to e.g. 10% of the cost in iii). Report the results of the best classifier.

Summarize briefly your findings from a) to d).

*Comments*: 

1) Description of the data set: https://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite)

2) For scaling the data, see: `preprocessing.StandardScaler` in scikit.

3) The parameters $C$ and $\gamma$ in SVMs are called *hyper-parameters* since they are considered to be fixed during optimization of the model parameters ($\theta$-values). The procedure of selecting hyper-parameter values is referred to as *model selection* and is typically performed by training the model for several different values of hyper-parameters, and evaluating the resulting model on a (cross-)validation set and finally selecting the values that give the best results.

4) An exhaustive search of parameter combinations on a grid is called is referred to as a *grid-search*. The `GridSearchCV` class in scikit-learn makes this easy to do. For details see the scikit documentation, sections 3.1 (Cross-validation: evaluating estimator performance), 3.2 (Tuning the hyper-parameters of an estimator) and the "Parameter estimation using grid search with cross-validation" example.

5) The paper *Random Search for Hyper-Parameter Optimization* by Bergstra and Bengio describes why randomized search of hyperparamter values is an efficient strategy. https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf


In [26]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler as std_scl
from sklearn.model_selection import GridSearchCV


#init train
trainData = np.genfromtxt(fname = "sat.trn")
X_train = trainData[:,:-1]
y_train = trainData[:,-1]

n,p = X_train.shape

X_train = np.c_[np.ones(n),X_train]

#init test
testData = np.genfromtxt(fname = "sat.tst")
X_test = testData[:,:-1]
y_test = testData[:,-1]

n_test,p_test = X_test.shape

X_test = np.c_[np.ones(n_test),X_test]


#Evaluate the accuracy of a RBF-SVM obtained with default values

ev = SVC().fit(X_train,y_train)
print("train set accuracy with default values of C and 𝛾 = ", ev.score(X_train, y_train))
print("test set accuracy with default values of C and 𝛾 = ",ev.score(X_test, y_test))


#scale the data
X_train_scale = std_scl().fit(X_train).transform(X_train)
X_test_scale = std_scl().fit(X_test).transform(X_test)



#Evaluate the accuracy of a RBF-SVM with scaled data
ev = SVC().fit(X_train_scale,y_train)
print("scaled train set accuracy with default values of C and 𝛾 = ", ev.score(X_train_scale, y_train))
print("scaled test set accuracy with default values of C and 𝛾 = ", ev.score(X_test_scale, y_test))


#cross validating to find C and 𝛾
grid = {
    'C': [1, 10],
    'gamma': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,
              0.7, 0.8, 0.9, 0.95, 0.99]
}

grid_search_cv = GridSearchCV(estimator = ev, param_grid = grid, cv = 3)
grid_search_cv = grid_search_cv.fit(X_train_scale, y_train)

print("here",grid_search_cv.best_score_)















train set accuracy with default values of C and 𝛾 =  0.8962795941375423
test set accuracy with default values of C and 𝛾 =  0.88
scaled train set accuracy with default values of C and 𝛾 =  0.9095828635851184
scaled test set accuracy with default values of C and 𝛾 =  0.8895


KeyboardInterrupt: 

### 2\. [Random Forests and feature selection, 20 points]
Quantitative Structure - Activity Relationship (QSAR) models relate the activity of chemical compounds to their structural properties. The activity can represent e.g. the potency of a drug or its toxicity. The structural properties may contain basic information such as i) the fraction of carbon atoms in the compound, ii) the spatial arrangement of atoms in the compound and iii) quantities computed from quantum mechanical simulations (*ab-initio* calculations).

The QSAR biodegradation Data Set `biodeg.csv` contains 41 molecular descriptors for two groups of compounds, those that are readily biodegradable (RB) and those that are not (NRB). The data set has 356 examples in the RB class and 699 in the NRB class, i.e. it is somewhat unbalanced.

a) Using `sklearn.ensemble.RandomForestClassifier`, obtain a classifier for predicting whether a given compound is readily biodegradable or not. Use a random 80/20 train/test set split for evaluting the performance of your classifier. Report the confusion matrix for the test set and calculate the accuracy of the classifier together with *sensitivity* and *specificity* (see below).

b) List the names of the 10 *most useful* features for the classification task (see below). Retrain a Random forests classifier using only the 10 most useful features and report sensitivity, specificity and accuracy. How does the performance compare to the classifier trained on the full feature set in a)?

*Comments*:

1) The file `biodeg_desc.txt` containes the feature names. A description of the data set can be found here: https://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation

2) Use `sklearn.model_selection.train_test_split` to create a train/test split.

3) A correctly predicted RB example is said to be a *true positive* and a correctly predicted NRB examples is said to be a *true negative*. When an NRB example is incorrectly predicted as RB it is a *false positive*. False negatives are defined analogously.

The *sensitivity* of a binary classifier is defined as TP/(TP+FN) and the *specificity* is defined as TN/(TN+FP) where TP is the number of true positives etc. These values are conveniently obtained from a confusion matrix, e.g. via `sklearn.metrics.confusion_matrix`

4) The feature importance measure provided by the Random Forests implementation in scikit has significant drawbacks. Therefore you will be using a so-called permutation importance measure (see problem 1 for a description). This is implemented in `sklearn.inspection.permutation_importance`.

5) Sidenote: Repeatedly retraining a classifier with smaller and smaller number of top features until the validation error (or out-of-bag error) starts to increase can easily lead to overfitting.

In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd


#CreateMatrix(y_actual, y_predicted)
#loops through the y values and compares them
#returns two (nxn) confusion matrixes with count and ratio
def createMatrix(y, y_pred):
    total = len(y)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for i in range(len(y)):
        if y[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y[i] == 0 and y_pred[i] == 0:
            TN += 1
        if y[i] == 0 and y_pred[i] == 1:
            FP += 1
        if y[i] == 1 and y_pred[i] == 0:
            FN += 1
    
    matrix = np.array([[TP,FP],
                      [FN,TN]])
    
    
    matrixR = np.array([[float(TP/total),float(FP/total)],
                       [float(FN/total),TN/total]])
    
    return matrix,matrixR

#matrixCalc(matrix)
#calculates sensitivity, specifity, accuracy and precision of an matrix
#returns dict with keys as "sensitivity", "specificity", "accuracy" and "precision"
#with the calculations as values
def matrixCalc(matrix):
    TP = matrix[0,0]
    FP = matrix[0,1]
    FN = matrix[1,0]
    TN = matrix[1,1]
    
    calc = {}
    calc["sensitivity"] = float(TP/(TP+FN))
    calc["specificity"] = float(TN/(TN + FP))
    calc["accuracy"] = float((TP+TN)/(TP + TN + FP + FN))
    calc["precision"] = float(TP/(TP + FP))
    
    return calc

#Insert data
Data = np.genfromtxt(fname = "biodeg.csv", delimiter = ';')
y = Data[:,-1]
X = Data[:,0:-1]
n,r = X.shape
X = np.c_[np.ones(n),X]

#classifying
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
random_forest_classifier = RandomForestClassifier().fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_test)

#finding accuracy

confMatrix, confMatrixR = createMatrix(y_test, y_pred)

#drawing the matrix
data = {'actual value 1':confMatrix[:,0],
       'actual value 0':confMatrix[:,1]}

dataR = {'actual value 1':confMatrixR[:,0],
       'actual value 0':confMatrixR[:,1]}

table = pd.DataFrame(data, index=['predicted value 1', 'predicted value 0'])
print("confusion matrix")
print(table,'\n\n')
tableR = pd.DataFrame(dataR, index=['predicted value 1', 'predicted value 0'])
print("confusion matrix ration")
print(tableR,'\n\n')


#defining value for calculations
calc = matrixCalc(confMatrix)
print([x + "=" + str(calc[x]) for x in calc])



confusion matrix
                   actual value 1  actual value 0
predicted value 1              61              10
predicted value 0              14             126 


confusion matrix ration
                   actual value 1  actual value 0
predicted value 1        0.289100        0.047393
predicted value 0        0.066351        0.597156 


['sensitivity=0.8133333333333334', 'specificity=0.9264705882352942', 'accuracy=0.8862559241706162', 'precision=0.8591549295774648']


In [90]:
from sklearn.inspection import permutation_importance

top = permutation_importance(random_forest_classifier,X_train, y_train)

print(top['importances'])


[[0.         0.         0.         0.         0.        ]
 [0.00236967 0.007109   0.00947867 0.00592417 0.00473934]
 [0.00118483 0.0035545  0.00236967 0.00236967 0.00236967]
 [0.00236967 0.00236967 0.00118483 0.00118483 0.00236967]
 [0.         0.         0.         0.         0.        ]
 [0.         0.00236967 0.00118483 0.00118483 0.00118483]
 [0.         0.         0.         0.         0.        ]
 [0.00236967 0.00236967 0.00236967 0.00236967 0.00236967]
 [0.00473934 0.00473934 0.00592417 0.00473934 0.00473934]
 [0.00236967 0.00118483 0.00118483 0.00118483 0.00118483]
 [0.00118483 0.00118483 0.         0.00118483 0.        ]
 [0.00118483 0.00236967 0.00118483 0.00118483 0.        ]
 [0.00118483 0.0035545  0.0035545  0.00236967 0.00473934]
 [0.00236967 0.00236967 0.00236967 0.00236967 0.00236967]
 [0.00236967 0.00118483 0.00118483 0.00236967 0.00118483]
 [0.00236967 0.00236967 0.00473934 0.00236967 0.00236967]
 [0.00236967 0.00236967 0.00236967 0.00236967 0.00236967]
 [0.00118483 0

### 3\. [Stacked regression models, 30 points]
In this problem you will construct a *stacked* two-stage regression model for a subset of the Million Song Database (MSD). The data set contains audio features for approximately 500K songs. Each song is represented by 90 features describing its "timbre" that are derived from the sampled recordings. The task is to predict the release year of a song.

A two-stage stacking model has several regression models in stage 1, all trained on the same data set. Predictions from stage 1 models form a new (derived) data set which is used as input to a single regression model in stage 2. This model "blends" predictions from the stage 1 models to create a final prediction, hopefully more accurate than the individual stage 1 predictions.

Your stacked regression model will employ Lasso, ExtraTrees, Random Forests and Gradient boosted trees in stage 1 and a linear regression model in stage 2. Training and testing are performed as follows:

*Training*: Train each model on the training set, using default parameters to begin with, but increase the number of trees for Extra Trees and Random Forests. Construct a training data set for the stage 2 model by sending the *validation* set (not the original training set) through each of the stage 1 models, resulting in an `n_val` by 4 matrix $X_2$ of prediction values. Train a linear regression model for stage 2 on $(X_2, y_\text{val})$.

*Testing*: Send the test data though all the models in stage 1 to obtain an `n_test` by 4 matrix. The stage 2 linear regression model is used to predict the data in this matrix to obtain the final predictions.

Start by creating a histogram of the number of songs per year in the data set to obtain insight into how realistic this prediction task is.

a) [20 points] Report the mean-squared error of the individual stage 1 models on the test set and the corresponding $R^2$ coefficient. Construct the stacked regression model described above, report its mean-squared error and $R^2$ coefficient on the test set.

b) [10 points] Answer the following questions:

i) Are the individual models doing a good job on the prediction task? (Consider the root-mean squared error).

ii) Are the individual classifiers failing in some obvious way (failure modes)?

iii) Is the stacking procedure worth the extra effort in your opinion? Why or why not?

iv) Why is it not a good idea to use the original training set to construct the $X_2$ data set for the stage 2 regression model?

*Comments*:

1) Download the subset of the Million Song Databse from here (210 MB): http://archive.ics.uci.edu/ml/datasets/YearPredictionMSD# (mirror: https://notendur.hi.is/steinng/kennsla/2021/ml/data/YearPredictionMSD.zip)

2) Use the train, validation and test partitions of the data defined in `load_msd.py`

`import load_msd as lmsd
X_train, y_train, X_val, y_val, X_test, y_test = lmsd.get_data(ntrain=10000)`

3) For Extra Trees and Random Forests you can set `n_jobs=-1` to use multiple cores/processors for training and prediction.

In [ ]:
# Insert code here
# ...

### 4\. [Preprocessing, performance metrics, 30 points]
In this problem you will construct a predictive model for Telemarketing. The data comes from a telemarketing campaign in Portugal where the goal was to get clients to subscribe to long-term savings deposits. The predictive model is to be used to identify customers that are likely to subscribe, based on personal information, economic indicators, whether the client has been contacted before etc. This should make the campaign more effective and reduce marketing costs.

The data is in file `bank-additional-full.csv` with a brief description in `bank-additional-names.txt`. The data contains a mixture of continuous and categorical features, with categorical data in text format. Some preprocessing is therefore needed before you can use it with scikit-learn algorithms.

The data is time ordered which means that randomly splitting it into training and test sets amounts to peeking into the future and will provide too optimistic estimates of model performance. This is therefore not a suitable evaluation strategy. In situations like this, one uses historical data to train a model and predicts data from the current period. To simulate this scenario you will use the most recent (last) 4000 samples for testing. You then use the remaining samples for training (or a subset thereof).

Train a Random Forests or an Extra Trees classifier and evaluate it on the test set using an appropriate performance metric (see below). The selection of metric should take into account whether the classes are balanced or not, as well as the goal of the prediction task. Do you think your model would be useful in practice? Why or why not?

*Comments*:

1) This is an open ended problem. There is no single correct answer.

2) The data set is described in some detail here:
https://archive.ics.uci.edu/ml/datasets/bank+marketing
and a previous attempt at predictive modeling in: https://www.sciencedirect.com/science/article/pii/S016792361400061X

3) To convert the data into a matrix format suitable for scikit-learn, it is probably easiest to use the Python Data Analysis Library (pandas) package. You load the data using

```python
import pandas as pd
bank_df=pd.read_csv('bank-additional-full.csv',sep=';')```

You can iterate over the features using e.g.

```python
for col in bank_df.columns:
    if bank_df[col].dtype == object:
        print("Categorical: ",col)
    else:
        print("Numerical: ", col)```

The output variable (`y`) is 1 if a customer subscribes and 0 otherwise. 

Start by using only the numerical data. Then add the categorical data gradually. More data does not always help.

The simplest conversion of categories to integers is called label encoding. In pandas:

```python
bank_df['y']=bank_df['y'].astype('category')
bank_df['y']=bank_df['y'].cat.codes
y=bank_df['y'].values
bank_df=bank_df.drop('y',axis=1) # Remove output variable```

or using scikit-learn instead:

```python
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(bank_df['y'])[:,0]```

Label encoding of a feature assumes that the feature values have a natural ordering (are ordinal). This has some drawbacks as detailed in the lecture notes and one-hot-encoding is generally preferred. This is most conveniently done by using `pd.get_dummies` with `drop_first=True`. For this particular data set, direct application of one-hot-encoding does not necessarily improve performance.

4) In addition to `sklearn.metrics.confusion_matrix` which can be used to derive sensitivity, specificity and accuracy, the `sklearn.metrics.classification_report` class provides performance metrics called precision recall and F-score (see Wikipedia for details).

In [ ]:
# Insert code here
# ...